In [1]:
# conda install -c rapidsai -c nvidia -c conda-forge dask-cuda cudatoolkit=11.6
# nvidia-smi
# conda install -c menpo opencv
from sklearn.model_selection import train_test_split, KFold
import dask.dataframe as dd
import dask.multiprocessing
import dask
import numpy as np
from dask import delayed
import dask_image.imread
import dask_image.ndfilters
import dask_image.ndmeasure
import cv2
from glob import glob
import json
import dask.array as da
import time
import xgboost as xgb
import pandas as pd
import os
from dask import compute
from sklearn.metrics import f1_score
# f1_score(y_true, y_pred, average=[‘micro’, ‘macro’, ‘samples’,’weighted’ 중 하나 선택])
# import tensorflow as tf
from dask.distributed import Client, LocalCluster
from dask_cuda import LocalCUDACluster

path = os.getcwd()
csv_features = ['내부 온도 1 평균', '내부 온도 1 최고', '내부 온도 1 최저', '내부 습도 1 평균', '내부 습도 1 최고', 
                '내부 습도 1 최저', '내부 이슬점 평균', '내부 이슬점 최고', '내부 이슬점 최저']
# from dask.distributed import Client, LocalCluster
# # client.close()
# cluster = LocalCluster()
# client = Client(cluster)
# print(cluster,client)
# client.close()
# http://localhost:8787/status

# from dask_cuda import LocalCUDACluster
# from dask.distributed import Client
# client.close()
# # http://127.0.0.1:34497/status
# cluster = LocalCUDACluster()
# client = Client(cluster)
# print(cluster,client)

In [2]:
crop = {'1':'딸기','2':'토마토','3':'파프리카','4':'오이','5':'고추','6':'시설포도'}
disease = {'1':{'a1':'딸기잿빛곰팡이병','a2':'딸기흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '2':{'a5':'토마토흰가루병','a6':'토마토잿빛곰팡이병','b2':'열과','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '3':{'a9':'파프리카흰가루병','a10':'파프리카잘록병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '4':{'a3':'오이노균병','a4':'오이흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '5':{'a7':'고추탄저병','a8':'고추흰가루병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '6':{'a11':'시설포도탄저병','a12':'시설포도노균병','b4':'일소피해','b5':'축과병'}}
risk = {'0':'정상','1':'초기','2':'중기','3':'말기'}


# #ensemble_labler
# label_description = {}
# for key, value in disease.items():
#     label_description[f'{key}_00_0'] = f'{crop[key]}_정상'
#     for disease_code in value:
#         for risk_code in risk:
#             label = f'{key}_{disease_code}_{risk_code}'
#             label_description[label] = f'{crop[key]}_{disease[key][disease_code]}_{risk[risk_code]}'

label_description = {
"1_00_0" : "딸기", 
"2_00_0" : "토마토",
"2_a5_2" : "토마토_흰가루병_중기",
"3_00_0" : "파프리카",
"3_a9_1" : "파프리카_흰가루병_초기",
"3_a9_2" : "파프리카_흰가루병_중기",
"3_a9_3" : "파프리카_흰가루병_말기",
"3_b3_1" : "파프리카_칼슘결핍_초기",
"3_b6_1" : "파프리카_다량원소결필(N)_초기",
"3_b7_1" : "파프리카_다량원소결필(P)_초기",
"3_b8_1" : "파프리카_다량원소결필(K)_초기",
"4_00_0" : "오이",
"5_00_0" : "고추",
 "5_a7_2" : "고추_탄저병_중기",
 "5_b6_1" : "고추_다량원소결필(N)_초기",
"5_b7_1" : "고추_다량원소결필(P)_초기",
 "5_b8_1" : "고추_다량원소결필(K)_초기",
"6_00_0" : "시설포도",
"6_a11_1" : "시설포도_탄저병_초기",
 "6_a11_2" : "시설포도_탄저병_중기",
 "6_a12_1" : "시설포도_노균병_초기",
"6_a12_2" : "시설포도_노균병_중기",
 "6_b4_1" : "시설포도_일소피해_초기",
 "6_b4_3" : "시설포도_일소피해_말기",
"6_b5_1" : "시설포도_축과병_초기"   }


global ensemble_label_encoder
ensemble_label_encoder = {key:idx for idx, key in enumerate(label_description)}
ensemble_label_decoder = {val:key for key, val in ensemble_label_encoder.items()}

#crop_labler
crop_label_description = {}
for key, value in disease.items():
    crop_label_description[f'{key}'] = f'{crop[key]}'
    
global crop_label_encoder
crop_label_encoder = {key:idx for idx, key in enumerate(crop_label_description)}
crop_label_decoder = {val:key for key, val in crop_label_encoder.items()}


#disease_labler
disease_label_description = {}
for key, value in disease.items():
    disease_label_description[f'00'] = "정상"
    for disease_code,value in value.items():
        label = f'{disease_code}'
        disease_label_description[label] = f'{value}'

global disease_label_encoder
disease_label_encoder = {key:idx for idx, key in enumerate(disease_label_description)}
disease_label_decoder = {val:key for key, val in disease_label_encoder.items()}


#risk_labler
risk_label_description = {}
for key, value in risk.items():
    label = f'{key}'
    risk_label_description[label] = f'{value}'

global risk_label_encoder
risk_label_encoder = {key:idx for idx, key in enumerate(risk_label_description)}
risk_label_decoder = {val:key for key, val in risk_label_encoder.items()}





In [3]:
crop_label_description

{'1': '딸기', '2': '토마토', '3': '파프리카', '4': '오이', '5': '고추', '6': '시설포도'}

In [4]:
risk_label_description

{'0': '정상', '1': '초기', '2': '중기', '3': '말기'}

In [5]:
disease_label_description

{'00': '정상',
 'a1': '딸기잿빛곰팡이병',
 'a2': '딸기흰가루병',
 'b1': '냉해피해',
 'b6': '다량원소결핍 (N)',
 'b7': '다량원소결핍 (P)',
 'b8': '다량원소결핍 (K)',
 'a5': '토마토흰가루병',
 'a6': '토마토잿빛곰팡이병',
 'b2': '열과',
 'b3': '칼슘결핍',
 'a9': '파프리카흰가루병',
 'a10': '파프리카잘록병',
 'a3': '오이노균병',
 'a4': '오이흰가루병',
 'a7': '고추탄저병',
 'a8': '고추흰가루병',
 'a11': '시설포도탄저병',
 'a12': '시설포도노균병',
 'b4': '일소피해',
 'b5': '축과병'}

In [6]:
label_description

{'1_00_0': '딸기',
 '2_00_0': '토마토',
 '2_a5_2': '토마토_흰가루병_중기',
 '3_00_0': '파프리카',
 '3_a9_1': '파프리카_흰가루병_초기',
 '3_a9_2': '파프리카_흰가루병_중기',
 '3_a9_3': '파프리카_흰가루병_말기',
 '3_b3_1': '파프리카_칼슘결핍_초기',
 '3_b6_1': '파프리카_다량원소결필(N)_초기',
 '3_b7_1': '파프리카_다량원소결필(P)_초기',
 '3_b8_1': '파프리카_다량원소결필(K)_초기',
 '4_00_0': '오이',
 '5_00_0': '고추',
 '5_a7_2': '고추_탄저병_중기',
 '5_b6_1': '고추_다량원소결필(N)_초기',
 '5_b7_1': '고추_다량원소결필(P)_초기',
 '5_b8_1': '고추_다량원소결필(K)_초기',
 '6_00_0': '시설포도',
 '6_a11_1': '시설포도_탄저병_초기',
 '6_a11_2': '시설포도_탄저병_중기',
 '6_a12_1': '시설포도_노균병_초기',
 '6_a12_2': '시설포도_노균병_중기',
 '6_b4_1': '시설포도_일소피해_초기',
 '6_b4_3': '시설포도_일소피해_말기',
 '6_b5_1': '시설포도_축과병_초기'}

In [7]:

@delayed
def padding(data, array_len, col_len):
    pad = np.zeros((array_len, col_len))
    length = min(array_len, len(data))
    
    pad[:length] = data[:length]
    return pad

@delayed
def img_resize(img):
    img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32)/255
    return img


@delayed
def label_encoding(label):
    global ensemble_label_encoder
    encoded_label = ensemble_label_encoder[label]
    return encoded_label

@delayed
def label_encoding_crop(label):
    global crop_label_encoder
    encoded_label = crop_label_encoder[label]
    return encoded_label

@delayed
def label_encoding_disease(label):
    global disease_label_encoder
    encoded_label = disease_label_encoder[label]
    return encoded_label

@delayed
def label_encoding_risk(label):
    global risk_label_encoder
    encoded_label = risk_label_encoder[label]
    return encoded_label

def getlable(jsonpath,type):
    if type == "ensemble":
        with open(jsonpath, 'r') as f:
            json_file = json.load(f)

        crop = json_file['annotations']['crop']
        disease = json_file['annotations']['disease']
        risk = json_file['annotations']['risk']
        label = f'{crop}_{disease}_{risk}'
        return label

    elif type == "crop":
        with open(jsonpath, 'r') as f:
            json_file = json.load(f)
        crop = json_file['annotations']['crop']
        label = f'{crop}'
        return label

    elif type == "dc":
        with open(jsonpath, 'r') as f:
            json_file = json.load(f)
        disease = json_file['annotations']['disease']
        label = f'{disease}'
        return label

    elif type == "risk":  
        with open(jsonpath, 'r') as f:
            json_file = json.load(f)
        risk = json_file['annotations']['risk']
        label = f'{risk}'
        return label

    
def get_label_list(labelpath_list):
    labelarr = np.array([])
    labelarr_crop = np.array([])
    labelarr_dc = np.array([])
    labelarr_risk = np.array([])


    # labelarr = da.array([])
    for ind,json_path in enumerate(labelpath_list):
        # label = label_encoding(getlable(json_path))
        # label = da.array(np.array(label_encoding(getlable(json_path))))
        label = np.array(label_encoding(getlable(json_path,"ensemble")))
        labelarr = np.append(labelarr,label)

        label_crop = np.array(label_encoding_crop(getlable(json_path,"crop")))
        labelarr_crop = np.append(labelarr_crop,label_crop)

        label_dc = np.array(label_encoding_disease(getlable(json_path,"dc")))
        labelarr_dc = np.append(labelarr_dc,label_dc)

        label_risk = np.array(label_encoding_risk(getlable(json_path,"risk")))
        labelarr_risk = np.append(labelarr_risk,label_risk)


    return labelarr, labelarr_crop, labelarr_dc, labelarr_risk

@delayed
def imageresize(img):
    # img = dask.delayed(cv2.resize)(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    # img = dask.delayed(img.astype(np.float32)/255)  ##픽셀값을 0~1사이로 정규화
    # # img = np.transpose(img, (2,0,1))
    # return dask.delayed(img.reshape)(-1,224,224,3)
    img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32)/255  ##픽셀값을 0~1사이로 정규화
    # img = np.transpose(img, (2,0,1))
    return np.array(img)

@delayed
def imageresize2(img):
    # img = dask.delayed(cv2.resize)(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    # img = dask.delayed(img.astype(np.float32)/255)  ##픽셀값을 0~1사이로 정규화
    # # img = np.transpose(img, (2,0,1))
    # return dask.delayed(img.reshape)(-1,224,224,3)
    img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    # img = img.astype(np.float32)/255  ##픽셀값을 0~1사이로 정규화
    # img = np.transpose(img, (2,0,1))
    return np.array(img)

In [8]:
@delayed
def get_all_image(imgpath_list,image_size):
    imgarr = np.empty((0,image_size,image_size,3), float)
    for ind,img_path in enumerate(imgpath_list):
        imgarr = delayed(np.append)(imgarr,img, axis = 0)
    return imgarr

In [9]:
%%time
label_list = sorted(glob(os.path.join(path,"data","train","*","*.json")))
y_train,y_train_crop,y_train_dc,y_train_risk, = get_label_list(label_list)
# y_train = y_train.rechunk(5767)
results = dask.compute(*y_train)
label = np.array(results)
# print(label.shape)
results_crop = dask.compute(*y_train_crop)
label_crop = np.array(results_crop)
#
results_dc = dask.compute(*y_train_dc)
label_disease = np.array(results_dc)
#
results_risk = dask.compute(*y_train_risk)
label_risk = np.array(results_risk)
label.shape,label_crop.shape,label_disease.shape,label_risk.shape

CPU times: user 4.12 s, sys: 274 ms, total: 4.4 s
Wall time: 5.2 s


((5767,), (5767,), (5767,), (5767,))

## 특징점찾기 예시

In [45]:
%%time
#making_img_set
row_img = dask_image.imread.imread(os.path.join(path,"data","train","*","*.jpg"))
train = [imageresize2(img) for img in row_img]
train_x = da.array(dask.compute(*train))
# os.path.join(path,"data","train","*","*.jpg")


CPU times: user 43.2 s, sys: 3.48 s, total: 46.6 s
Wall time: 13.3 s


In [46]:
for i in range(len(train_x)):
    image = train_x[i].compute()[:,:,:]
    # gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    a = featureextrating(image).compute()
    print(a)


[[ 17.   9.  13. ...  42.   8.   0.]
 [ 13.   9.   2. ... 112.  40.  18.]
 [  2.  34.  26. ...   1.   0.   0.]
 ...
 [  0.   3.  56. ...  25.   6.   1.]
 [ 43.  30.  10. ...  20.   0.   0.]
 [ 20.  17.   4. ...  26.   0.   0.]]
[[  6.   2.   1. ... 111.  25.   0.]
 [  1.   4.   3. ...   0.   0.   5.]
 [  0.   0.   0. ...   1.   2.  17.]
 ...
 [ 16.   4.   2. ...   1.   4.  45.]
 [  0.   0.   1. ...   0.   0.   0.]
 [  5.   0.   0. ...   4.  10.   0.]]
[[ 0.  0.  0. ...  1.  3.  1.]
 [19. 44.  4. ...  0.  1. 43.]
 [44.  6.  1. ...  0.  1.  6.]
 ...
 [ 0.  0.  0. ... 65.  0.  0.]
 [ 5.  1.  2. ... 10.  0.  0.]
 [ 7.  1.  5. ...  2.  3.  2.]]
[[ 42.  95. 129. ...  34.   4.   0.]
 [  0.   0.   1. ...  51.   1.   4.]
 [ 36.   4.   0. ...  20.  73.  25.]
 ...
 [ 60.  28.   5. ...   2.   4.  82.]
 [ 35.   8.   0. ...  26.   2.   5.]
 [ 22.  10.   9. ...   0.   0.   0.]]
[[  0.   0.   0. ...   0.   0.   3.]
 [  2.   6.   3. ...   0.   0.   0.]
 [  9. 112.  98. ...   1.   0.   0.]
 ...
 [  1.  

KeyboardInterrupt: 

In [14]:
sift = cv2.xfeatures2d.SIFT_create(300)
keypoints, descriptor = sift.detectAndCompute(gray,None)
print("keypoint: ", len(keypoints)," descriptor: ", descriptor.shape)
print(descriptor)

keypoint:  300  descriptor:  (300, 128)
[[ 1. 14. 54. ...  0.  0.  0.]
 [41.  8.  8. ...  0.  0.  3.]
 [29. 57.  2. ... 11.  6.  4.]
 ...
 [ 6. 36. 70. ...  0.  0.  2.]
 [ 5. 13. 15. ... 10. 14.  9.]
 [ 2.  6. 19. ...  0.  0.  0.]]


## 이미지 매칭해보기

In [13]:
# # match 함수로부터 올바른 매칭점 찾기 (match_good.py)

# # import cv2, numpy as np

# img1 = train_x[0].compute()
# img2 = train_x[1].compute()
# gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
# gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# # ORB로 서술자 추출 ---①
# detector = cv2.ORB_create()
# kp1, desc1 = detector.detectAndCompute(gray1, None)
# kp2, desc2 = detector.detectAndCompute(gray2, None)
# # BF-Hamming으로 매칭 ---②
# matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
# matches = matcher.match(desc1, desc2)

# # 매칭 결과를 거리기준 오름차순으로 정렬 ---③
# matches = sorted(matches, key=lambda x:x.distance)
# # 최소 거리 값과 최대 거리 값 확보 ---④
# min_dist, max_dist = matches[0].distance, matches[-1].distance
# # 최소 거리의 15% 지점을 임계점으로 설정 ---⑤
# ratio = 0.2
# good_thresh = (max_dist - min_dist) * ratio + min_dist
# # 임계점 보다 작은 매칭점만 좋은 매칭점으로 분류 ---⑥
# good_matches = [m for m in matches if m.distance < good_thresh]
# print('matches:%d/%d, min:%.2f, max:%.2f, thresh:%.2f' \
#         %(len(good_matches),len(matches), min_dist, max_dist, good_thresh))
# # 좋은 매칭점만 그리기 ---⑦
# res = cv2.drawMatches(img1, kp1, img2, kp2, good_matches, None, \
#                 flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
# # 결과 출력
# cv2.imshow('Good Match', res)
# cv2.waitKey()
# cv2.destroyAllWindows()

## 특징점으로 예측모델

In [63]:
%%time
descriptor_n = 100

@delayed
def featureextrating(image):
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    sift = cv2.xfeatures2d.SIFT_create(descriptor_n)
    keypoints, descriptor = sift.detectAndCompute(gray,None)
    # print("keypoint: ", len(keypoints)," descriptor: ", descriptor.shape)
    # print(descriptor)
    return descriptor
    
#making_img_set
row_img = dask_image.imread.imread(os.path.join(path,"data","train","*","*.jpg"))
train = [imageresize2(img) for img in row_img]
# train = [featureextrating(imageresize(img)) for img in row_img]
train_x = da.array(dask.compute(*train))
# os.path.join(path,"data","train","*","*.jpg")
train_x_feature = da.zeros((len(train_x),descriptor_n,128))

for i in range(len(train_x)):
    print(i)
    image = train_x[i].compute()[:,:,:]
    # gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    train_x_feature[i] = featureextrating(image).compute()


0


ValueError: Can't broadcast data with shape (101, 128) across shape (100, 128)

In [68]:
featureextrating(image).compute()

array([[ 7.,  1.,  0., ...,  0.,  0.,  2.],
       [ 1.,  2.,  4., ..., 21.,  3.,  0.],
       [ 3.,  2.,  2., ...,  1.,  2.,  9.],
       ...,
       [38., 50.,  9., ..., 76.,  5.,  1.],
       [ 6.,  5., 41., ...,  2., 12., 68.],
       [ 9., 23., 30., ...,  3., 59., 23.]], dtype=float32)

In [57]:
featureextrating(image).compute()

array([[ 0.,  2., 15., ...,  0.,  0.,  0.],
       [ 2.,  1.,  3., ...,  0.,  0., 44.],
       [22.,  6.,  0., ...,  0.,  0.,  1.],
       ...,
       [ 7.,  1.,  0., ...,  0.,  0.,  0.],
       [ 8.,  3.,  0., ..., 54., 29.,  1.],
       [ 4.,  0.,  2., ...,  0.,  0.,  0.]], dtype=float32)

In [52]:
len(train_x)

5767

In [49]:
image = train_x[0].compute()[:,:,:]
# gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
a = featureextrating(image).compute()
print(a.shape)

(300, 128)


In [44]:
for i in range(len(train_x)):
    image = train_x[i].compute()[:,:,:]
    # gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    a = featureextrating(image).compute()
    print(a)


error: OpenCV(3.4.2) /tmp/build/80754af9/opencv-suite_1535558553474/work/opencv_contrib-3.4.2/modules/xfeatures2d/src/sift.cpp:1121: error: (-5:Bad argument) image is empty or has incorrect depth (!=CV_8U) in function 'detectAndCompute'


In [39]:
cv2.cvtColor(train[1].compute()[:,:,:],cv2.COLOR_BGR2GRAY)

array([[204, 207, 213, ..., 135, 125, 120],
       [203, 206, 214, ..., 132, 124, 117],
       [205, 208, 215, ..., 130, 121, 115],
       ...,
       [200, 201, 201, ..., 219, 219, 219],
       [196, 197, 200, ..., 220, 219, 218],
       [193, 194, 196, ..., 218, 218, 218]], dtype=uint8)

In [41]:
featureextrating(train_x[1].compute()[:,:,:]).compute()

array([[  6.,   2.,   1., ..., 111.,  25.,   0.],
       [  1.,   4.,   3., ...,   0.,   0.,   5.],
       [  0.,   0.,   0., ...,   1.,   2.,  17.],
       ...,
       [ 16.,   4.,   2., ...,   1.,   4.,  45.],
       [  0.,   0.,   1., ...,   0.,   0.,   0.],
       [  5.,   0.,   0., ...,   4.,  10.,   0.]], dtype=float32)

(224, 224, 3)

In [ ]:
train_x


,Array,Chunk
Bytes,3.23 GiB,32.01 MiB
Shape,"(5767, 224, 224, 3)","(223, 112, 112, 3)"
Count,104 Tasks,104 Chunks
Type,float32,numpy.ndarray


In [ ]:
X_train = train_x.reshape(5767,-1)

In [ ]:
X_train

dask.array<reshape, shape=(5767, 150528), dtype=float32, chunksize=(223, 37632), chunktype=numpy.ndarray>

In [ ]:
X, X_test, y, y_test,y_crop, y_crop_test,y_risk, y_risk_test,y_disease, y_disease_test, = train_test_split(X_train, label,label_crop,label_risk,label_disease, test_size=0.2, shuffle= True,stratify=label)

/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/sklearn/utils/__init__.py:202: PerformanceWarning: Slicing with an out-of-order index is generating 170 times more chunks
  return array[key] if axis == 0 else array[:, key]
/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/sklearn/utils/__init__.py:202: PerformanceWarning: Slicing with an out-of-order index is generating 43 times more chunks
  return array[key] if axis == 0 else array[:, key]
